In [10]:
import Pkg
Pkg.add("Oscar")
Pkg.add("Nemo")
Pkg.add("Polymake")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [11]:
using Oscar
using Nemo
using Polymake

In [248]:
function coneMultiplicity(a,b)
    return dot(a, [1,1,1,1])
end

coneMultiplicity (generic function with 1 method)

In [252]:
function compareCones(cone1, cone2, rayMatrix, distinguished)
    # Calculate the number of non-distinguished rays
    nondist1 = count(cone1) - dot(cone1, distinguished)
    nondist2 = count(cone2) - dot(cone2, distinguished)
    if (nondist1 - nondist2 != 0)
        return nondist1 - nondist2
    else
        # Need to use the method for calculating multiplicity of cone
        mult1 = coneMultiplicity(cone1, rayMatrix)
        mult2 = coneMultiplicity(cone2, rayMatrix)
        return mult1 - mult2
    end
end

compareCones (generic function with 1 method)

In [253]:
function extremalCones(coneList, rayMatrix, distinguished)
    # Finds the extremal cones according to # distinguished rays and multiplicity
    # distinguished is a boolean vector whose size is equal to the number of rays
    # The i-th index is 1 if the i-th ray (in rayMatrix) is distinguished
    maxCones = [coneList[1,:]]
    for i in 1:size(coneList,1)
        cone = coneList[i,:]
        # Compare the cone with the first element of the maximal cone list
        comp = compareCones(cone, maxCones[1], rayMatrix, distinguished)
        if (comp > 0)
            maxCones = [cone]
        elseif (comp == 0)
            push!(maxCones, cone)
        end
    end
    return maxCones
end

maximalCones (generic function with 2 methods)

In [254]:
maximalCones(coneList, rayMatrix, Bool[0,0,0,1])

2-element Vector{BitVector}:
 [1, 1, 0, 0]
 [0, 1, 1, 0]

In [14]:
R = ZZ
S = MatrixSpace(R, 3, 6)
(dim, c) = Nemo.nullspace(S([5 0 0 6 6 10; 0 9 1 6 4 7; 0 0 4 6 3 7]))

(3, [758890 -93990 -12; 316184 -39160 -5; 948503 -117474 -15; -632263 78307 10; -147 18 0; 1 0 0])

In [108]:
# Same function as in makeSimplicial
function getConeRays(coneRayIndices, rayMatrix)
    return convert(Matrix{Int}, tv.RAYS[findall(bool -> bool, coneRayIndices),:])
end

getConeRays (generic function with 1 method)

In [162]:
function findStackyPoint(ray, cone, rayMatrix, stack)
    # ray is the given "black" lattice point
    # stackyCone and rayMatrix are the rays of the cone
    # which should also contain information about the red lattice
    # Returns the first "red" lattice point along the given ray
    # Given the rays of stackyCone, \rho_i and \delta_i, find
    # \psi = a_1 \rho_1 + ... + b_1 \delta_1 + ... + b_n \delta_n
    # and return a_1, ..., b_1, ..., b_n, and the multiple of ray
    
    # Apply stacky structure
    rays = getConeRays(cone, rayMatrix) .* transpose(stack)
    # Find integer solutions
    M = hcat(rays, ray)
    S = MatrixSpace(ZZ, size(M,1), size(M,2))
    (dim, c) = nullspace(S(M))
    # Get smallest integer multiple
    coef = nothing
    for col in 1:size(c, 2)
        if (c[size(c, 1), col] != 0)
            coef = vec(Matrix{Int}(transpose(c[1:size(c,1), col]) * sign(-c[size(c, 1), col])))
            coef[size(coef, 2)] = abs(coef[size(coef, 2)])
        end
    end
    # Return the coefficients of the rays of the cone, in the same order
    return coef
end

findStackyPoint (generic function with 1 method)

In [77]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [78]:
cone = tv.MAXIMAL_CONES[1,:]

4-element BitVector:
 1
 1
 0
 0

In [79]:
rayMatrix = tv.RAYS

pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [163]:
# Should output 6, the first red point along [1,1], 3,2
coef = findStackyPoint([1,1], cone, rayMatrix, [2,3])

3-element Vector{Int64}:
  3
  2
 -6